In [40]:
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8")

In [48]:
mnist_trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=None)

print(mnist_trainset[1])

(<PIL.Image.Image image mode=L size=28x28 at 0x7FC2C4581FC0>, 0)


In [42]:
class ConvNN(nn.Module):
    
    def __init__(self, channels, kernels, layers):
        super(ConvNN, self).__init__()
        
        self.conv = nn.ModuleList([nn.Conv2d(c, channels[i+1], k) for (i, c), k in zip(enumerate(channels[:-1]), kernels)])
        self.linear = nn.ModuleList([nn.Linear(v, layers[i+1]) for i, v in enumerate(layers[:-1])])
        self.pool = nn.MaxPool2d(2, 2)
    
    
    def forward(self, x):    
        for l in self.conv:
            x = self.pool(
                F.relu(
                l(x)
            ))
        
        x = torch.flatten(x, 1)
        
        for l in self.linear[:-1]:
            x = F.relu(l(x))
        
        x = nn.Softmax(self.linear[-1](x))
        
        return torch.argmax(x)
        
    

In [44]:
device = "cuda" if torch.cuda.is_available() else "cpu"

NUM_EPOCHS = 1000
LEARNING_RATE = 0.001


model = ConvNN([1, 5, 3], [5, 3], [28 * 28 * 3 * 3, 512, 10]).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

losses = []

for epoch in range(NUM_EPOCHS):
    
    optimizer.zero_grad()
    outputs = model(mnist_trainset)
    loss = criterion(outputs, mnist_trainset.targets)
    
    loss.backward()
    optimizer.step()
    
    losses.append(loss.item())
    
print(f"Final loss = {losses[-1]}")
plt.plot(range(NUM_EPOCHS), losses)
plt.show()

IndexError: too many indices for tensor of dimension 1